In [1]:
using NCDatasets
using Statistics
using Base.Threads

function compute_pixelwise_mean(input_path::String, output_path::String, varname::String;
    nan_flags = [200, 201, 211, 237, 239, 250, 254, 255])

    println("Opening input dataset...")
    ds = NCDataset(input_path, "r")
    data = ds[varname]
    size_x, size_y, size_t = size(data)

    println("Creating output dataset...")
    ds_out = NCDataset(output_path, "c")

    defDim(ds_out, "x", size_x)
    defDim(ds_out, "y", size_y)
    defDim(ds_out, "time", 1)

    outvar = defVar(ds_out, "mean_value", Float32, ("x", "y", "time"); deflatelevel=4)
    outvar.attrib["units"] = "fraction"

    println("Beginning threaded pixel processing...")

    # Create a shared output variable
    Threads.@threads for j = 1:size_y
        row_out = Array{Float32}(undef, size_x)
        buf = Array{Float32}(undef, size_t)

        for i = 1:size_x
            # Handle missing values with NaN fill
            buf .= coalesce.(data[i, j, :], NaN32)

            nan_count = 0
            for k = 1:size_t
                val = buf[k]
                if val in nan_flags
                    nan_count += 1
                    buf[k] = (60 <= k <= 210) ? 1.0 : 0.0
                else
                    buf[k] = val / 100.0
                end
            end

            row_out[i] = (nan_count > 0.75 * size_t) ? NaN32 : mean(buf)
        end

        # Thread-safe write: one thread per row
        outvar[:, j, 1] = row_out

        if j % 100 == 0
            @info "Thread $(threadid()): completed row $j / $size_y"
        end
    end

    close(ds)
    close(ds_out)

    println("All done. Output written to: $output_path")
end


compute_pixelwise_mean (generic function with 1 method)

In [2]:
Threads.nthreads()   # should show >1


8

In [ ]:
compute_pixelwise_mean("/Users/jldechow/Documents/Projects/UNC/CoReSSD/SCF.nc", "/Users/jldechow/Documents/Projects/UNC/CoReSSD/meanSCF.nc", "SCF")


Opening input dataset...
Creating output dataset...
Beginning threaded pixel processing...
